# Jscatter : Build Models

This Notebook aims to demonstrate how ot build your own models for fitting.

### Content
- How to build simple models
- How to build a complex model

![Jscatter](https://jscatter.readthedocs.io/en/latest/_images/Jscatter.jpeg "Jscatter")

#### Prerequisite

In [ ]:
import sys
!{sys.executable} -m pip install jscatter          
%matplotlib notebook

import jscatter as js
import numpy as np
js.usempl(True)   # force matplotlib, not needed in Linux

## How to build simple models
... which are actually not so simple....

Models get as input a number of parameters with one as the X values and 
should return only Y values or a dataArray with `.Y`. 

Except the X values all other parameters should be single valued (no list).
Use numpy ufunctions as shown below.

### Build models in one line using lambda
Directly calc the values and return only Y values

In [ ]:
diffusion = lambda A,D,t,wavevector,elastic=0: A*np.exp(-wavevector**2*D*t)+elastic

### Use a model from the libraries
Here Teubner-Strey model is used adding background and power law.

This model returns as above only Y values.

In [ ]:
tbpower = lambda q,B,xi,dd,A,beta,bgr: js.ff.teubnerStrey(q=q, xi=xi, d=dd).Y*B+A*q**beta+bgr

### The same as above in a function definition
Function definitions are the prefered way as they are more clean, e.g. they have a defined function name.

In [ ]:
def diffusion2(A, D, t, elastic, wavevector=0):
    Y = A * np.exp(-wavevector ** 2 * D * t) + elastic
    return Y

### Returning dataArray allows additional attributes to be included in the result
This returns a dataArray with X, Y values and attributes.
The attributes as e.g. the chosen parameters for the calculation are documented in this way and will not be forget.

In [ ]:
def diffusion3(A, D, t, wavevector, elastic=0):
    # calc values
    Y = A * np.exp(-wavevector ** 2 * D * t) + elastic
    # create dataArray from calculated values
    result = js.dA(np.c_[t, Y].T)
    # add attributes
    result.diffusioncoefficient = D
    result.wavevector = wavevector
    result.columnname = 'time;Iqt'
    return result

In [ ]:
def tbpower2(q, B, xi, dd, A, beta, bgr):
    """Model Teubner Strey  + power law and background. 
    This string will be later a docstring and describe the model"""
    # save different contributions for later analysis
    tb = js.ff.teubnerStrey(q=q, xi=xi, d=dd)  # this returns a dataArray
    pl = A * q ** beta                         # power law
    tb = tb.addZeroColumns(2)
    tb[-2] = pl                                # save power law in second last column
    tb[-1] = tb.Y                              # save Teubner-Strey in last column
    # put full model to Y values (usually tb[1])
    tb.Y = B * tb.Y + pl + bgr  
    # save the additional parameters ; xi and d already included in teubnerStrey
    tb.A = A
    tb.bgr = bgr
    tb.beta = beta
    # name the columns 
    tb.columnname = 'q;Iq,IqTb,Iqpower'
    # set no errors for Y 
    tb.setColumnIndices(iey=None)
    return tb


## How to build a complex model

Build a complex model of different components.

The docstring (in tripple "") can be used to generate the documentation. The used style corresponds to the numpy type docstring with specific sections as Parameters, Returns, Notes, Examples and References.

In [ ]:
def particlesWithInteraction(q, Ra, Rb, molarity, bgr, contrast, collimation=None, beta=True):
    """
    Particles with interaction and ellipsoid form factor as a model for e.g. dense protein solutions.

    Document your model if needed for later use that you know what you did and why.
    Or make it short without all the nasty documentation for testing.
    The example neglects the protein exact shape and non constant scattering length density.
    Proteins are more potato shaped  and nearly never like a ellipsoid or sphere.
    So this model is only valid at low Q as an approximation.

    Parameters
    ----------
    q : float
        Wavevector
    Ra,Rb : float
        Radius
    molarity : float
        Concentration in mol/l
    contrast : float
        Contrast between ellipsoid and solvent.
    bgr : float
        Background e.g. incoherent scattering
    collimation : float
        Collimation length for SANS. For SAXS use None.
    beta : bool
        True include asymmetry factor beta of
        M. Kotlarchyk and S.-H. Chen, J. Chem. Phys. 79, 2461 (1983).

    Returns
    -------
        dataArray

    Notes
    -----
    Explicitly:
    **The return value can be a dataArray OR only Y values**. Both is working for fitting.
    Reference can be included from below as [1]_ .

    Examples
    --------
    ::
    
     import jscatter as js
     # do something
     
    References
    ----------
    .. [1] Another one bites the dust
           Queen, Album: The Game (1980)
           

    """
    # We need to multiply form factor and structure factor and add an additional background.
    # formfactor of ellipsoid returns dataArray with beta at last column.
    ff = js.ff.ellipsoid(q, Ra, Rb, SLD=contrast)
    V = ff.EllipsoidVolume
    # the structure factor returns also dataArray
    # we need to supply a radius calculated from Ra Rb, this is an assumption of effective radius for the interaction.
    R = (Ra * Rb * Rb) ** (1 / 3.)
    # the volume fraction is concentration * volume
    # the units have to be converted as V is usually nm**3 and concentration is mol/l
    sf = js.sf.PercusYevick(q, R, molarity=molarity)
    if beta:
        # beta is asymmetry factor according to M. Kotlarchyk and S.-H. Chen, J. Chem. Phys. 79, 2461 (1983).
        # correction to apply for the structure factor
        # noinspection PyProtectedMember
        sf.Y = 1 + ff._beta * (sf.Y - 1)
    #
    # molarity (mol/l) with conversion to number/nm**3 result is in cm**-1
    ff.Y = molarity * 6.023e23 / (1000 * 1e7 ** 2) * ff.Y * sf.Y + bgr
    # add parameters for later use; ellipsoid parameters are already included in ff
    # if data are saved these are included in the file as documentation
    # or can be used for further calculations e.g. if volume fraction is needed (V*molarity)
    ff.R = R
    ff.bgr = bgr
    ff.Volume = V
    ff.molarity = molarity
    # for small angle neutron scattering we may need instrument resolution smearing
    if collimation is not None:
        # For SAX we set collimation=None and this is ignored
        # For SANS we set some reasonable values as 2000 (mm) or 20000 (mm)
        # as attribute in the data we want to fit.
        result = js.sas.resFunct(ff, collimation, 10, collimation, 10, 0.7, 0.15)
    else:
        result = ff
    # we return the complex model for fitting
    return result

### Use the model to simulate and plot this 
Hint for fitting SANS data or other fits combining different measurements described by a common parameter.

For a combined fit of several collimation distances each dataset should contain an attribute data.collimation.
This is automatically used in the fit, if there is not explicit fit parameter with this name. SAXS data may not have this parameter or set it to zero.
In this way different methods can be discriminated in the model function. Also char parameters are allowed as fixed parameters.

In [ ]:
p = js.mplot()
q = js.loglist(0.1, 5, 300)
for m in [0.01, 0.1, 1, 2, 3, 4]:
    data = particlesWithInteraction(q, Ra=3, Rb=4, molarity=m * 1e-3, bgr=0, contrast=1)
    p.Plot(data, sy=[-1, 0.3, -1], le='c= $molarity M')

p.Legend()
p.Yaxis(min=100, max=1e9, scale='l', label='I(Q) / $cm^{-1}$', tick=[10, 9])
p.Xaxis(scale='n', label='Q / $nm^{-1}$')
p.Title('Ellipsoidal particles with interaction\nRa=3, Rb=4 and PercusYevick structure factor')
